# Selecting Rows with Core

Penggunaan fungsi <b>select()</b> sama seperti penggunaan fungsi insert().

In [1]:
from sqlalchemy import create_engine, MetaData

engine = create_engine("sqlite+pysqlite:///example.db", future=True)
metadata = MetaData(bind=engine)
metadata.reflect()

users_table = metadata.tables["users"]
email_table = metadata.tables["email"]

In [2]:
from sqlalchemy import select
sel_stmt = select(users_table)
print(sel_stmt)

SELECT users.id, users.name, users.fullname 
FROM users


In [3]:
sel_stmt = select(users_table).where(users_table.c.name =="spongebob")
print(sel_stmt)

SELECT users.id, users.name, users.fullname 
FROM users 
WHERE users.name = ?


In [4]:
with engine.connect() as conn:
    for row in conn.execute(sel_stmt):
        print(row)

(1, 'spongebob', 'spongebob squarepants')


In [5]:
with engine.connect() as conn:
    for row in conn.execute(select(users_table)):
        print(row)

(1, 'spongebob', 'spongebob squarepants')
(2, 'sandy', 'sandy cheeks')
(3, 'patrick', 'patrick star')


## SELECT Individual Columns

In [6]:
sel_stmt = select(users_table.c.name)
print(sel_stmt)

SELECT users.name 
FROM users


In [12]:
with engine.connect() as conn:
    result = conn.execute(sel_stmt)
    print(result.fetchall())

[('spongebob',), ('sandy',), ('patrick',)]


In [14]:
with engine.connect() as conn:
    result = conn.execute(select(email_table))
    print(result.all())

[(1, 1, 'spongebob@aol.com'), (2, 2, 'sandy@aol.com'), (3, 3, 'patrick@aol.com')]


## SELECT from two table

SELECT statement di bawah ini melakukan SELECT kolom name pada tabel users dan mengkombinasikan dengan seluruh kolom yang ada pada tabel email.

In [15]:
with engine.connect() as conn:
    result = conn.execute(
        select(users_table.c.name, email_table).
        where(users_table.c.id == email_table.c.user_id).
        order_by(email_table.c.id)
    )
    
    for row in result.all():
        print(row)

('spongebob', 1, 1, 'spongebob@aol.com')
('sandy', 2, 2, 'sandy@aol.com')
('patrick', 3, 3, 'patrick@aol.com')


## Selecting from Labeled SQL Expressions

In [20]:
sel_stmt = select(
               ("Username: " + users_table.c.name).label("username")
           ).order_by(users_table.c.name)
    
with engine.connect() as conn:
    for row in conn.execute(sel_stmt):
        print(f'{row.username}')

Username: patrick
Username: sandy
Username: spongebob


## Selecting with Textual Column Expressions

In [23]:
from sqlalchemy import text

sel_stmt = (
    select(
        text("'some phrase'"), users_table.c.name
    ).order_by(users_table.c.name)
)

with engine.connect() as conn:
    print(conn.execute(sel_stmt).all())

[('some phrase', 'patrick'), ('some phrase', 'sandy'), ('some phrase', 'spongebob')]
